# Récupération des notes données aux arrondissements de Paris sur ville-ideale.fr 

'''We want to scrap the marks that citizens of Paris have given to their arrondissement in the website ville-ideale.fr . The marks are out of 10 and the global mark is :(mean of 8 criterias + quality of life criteria) / 2  . '''

In [222]:
'''Importation of webscraping and datascience packages'''
#!pip install -q lxml
import numpy as np
import bs4 #BeautifulSoup4 for scraping
import lxml
import pandas #datascience library
import urllib #scraping
import time
import requests #scraping
import random
import yaml #to use yml files
from urllib import request #to get the html source code

In [225]:
'''We use a user-agent and other headers also to avoid being detected as a robot by the website'''
'''We open the file headers in which we have different headers (for Chrome,Firefox...) and save them in browser_headers'''
with open("headers.yml") as f_headers:
    browser_headers = yaml.safe_load(f_headers)

In [228]:
'''We first create lists for name of arrondissement and for postal codes to find the corresponding web page on the website ville-ideale.fr'''
arrondissements_list= ["1er"]+[str(i) + "e" for i in range(2,21)]
postal_codes_list=["751"+"0"+ str(i) if i in range(1,10) else "751"+str(i) for i in range(1,21)]


In [229]:
'''We create a list of the url of the web pages of each arrondissement on the website ville-ideale.fr'''
arrondissements_table=[[arrondissements_list[i],postal_codes_list[i]] for i in range(20)]
urls=["https://www.ville-ideale.fr/paris-"+arr[0]+"-arrondissement_"+ arr[1] for arr in arrondissements_table]
'''We initialise the final DataFrame that will contain all the marks for all arrondissements'''
df_marks_arrondissements=pandas.DataFrame()


In [231]:
'''We scrap the marks for each arrondissement thanks to a loop on the page urls of each arrondissement.
This code takes about one hour to run because of the several time.sleep() but it is necessary in order to not be blocked by the website'''
for i in range(20):
    headers=list(browser_headers.values())[random.randint(0,len(list(browser_headers.values()))-1)]
    '''Getting the source code of the arrondissement page'''
    url_arr = urls[i] #url of the arrondissement page to scrap
    request_text = requests.get(url_arr,headers=headers).text #getting the html source code
    time.sleep(30)
    '''Create a Python object from the web page with BeautifulSoup4 to facilitate html data retrieval'''
    page_arr = bs4.BeautifulSoup(request_text, "lxml")
    time.sleep(20)
    '''Getting the global mark of the arrondissement'''
    global_mark = page_arr.find('p', {'id' : 'ng'})
    time.sleep(40)
    '''Getting the table of marks for the 9 criterias + the global mark as a dataframe of one row and 10 columns'''
    table_marks = page_arr.find('table', {'id' : 'tablonotes'})
    time.sleep(60)
    rows_marks = table_marks.find_all('tr') #Getting the rows of the table
    time.sleep(20)
    df_marks=pandas.DataFrame()
    #We will create two lists: one for criterias and one for corresponding marks using each row of the table
    criteria_list=[]
    marks_list=[]
    for row in rows_marks:
        criteria_list.append(row.find('th').text.strip())
        marks_list.append(row.find('td').text.strip())
    
    df_marks=pandas.DataFrame(marks_list).transpose() #We create a 1 row dataframe with the marks for the 9 criterias
    df_marks.columns=criteria_list #We assign the names of the criterias to the columns
    df_marks["Note Globale"]=global_mark.text.strip()[:4] #We add the global mark (one column more)
    df_marks.index=["Paris "+arrondissements_list[i]] #We give the row the name of the arrondissement
    df_marks_arrondissements=pandas.concat([df_marks_arrondissements,df_marks])# We finally add this line to the final DataFrame
    time.sleep(random.randint(1,5))

## Nous obtenons ce tableau final des notes selon l'arrondissement et le critère :

In [232]:
'''We obtain this final table of marks for all arrondissements'''
df_marks_arrondissements

,Environnement,Transports,Sécurité,Santé,Sports et loisirs,Culture,Enseignement,Commerces,Qualité de vie,Note Globale
Paris 1er,"4,50","9,33","5,25","6,50","6,00","8,58","6,17","8,25","6,33","6,58"
Paris 2e,"3,13","8,17","6,38","7,25","4,63","8,00","5,67","8,46","6,00","6,23"
Paris 3e,"5,52","8,19","7,93","7,93","6,78","9,33","7,81","8,81","7,52","7,65"
Paris 4e,"4,27","7,62","6,69","6,46","5,12","7,46","7,12","6,77","5,73","6,09"
Paris 5e,"7,30","8,52","8,18","8,55","7,00","8,50","8,95","8,41","8,45","8,32"
Paris 6e,"6,25","8,60","7,90","8,10","5,90","8,40","8,55","8,20","7,55","7,65"
Paris 7e,"6,88","7,60","7,96","6,76","5,60","7,24","7,52","6,80","7,56","7,30"
Paris 8e,"6,78","9,00","7,83","7,50","5,50","7,44","7,67","7,94","8,06","7,76"
Paris 9e,"5,14","8,50","6,24","7,62","6,17","8,50","7,24","8,55","7,26","7,25"
Paris 10e,"3,24","7,63","3,74","6,60","5,75","6,18","5,53","6,51","4,54","5,10"


In [239]:
'''We save this dataframe in a excel file "Notes_arrondissements" '''
name="Notes_arrondissements.xlsx"
df_marks_arrondissements.to_excel(name)